In [13]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_curve, auc

from importlib import reload
import sys
sys.path.append("/home/ziweiji/Hallu_Det/src/")
import binary_threshold_utils
reload(binary_threshold_utils)
from binary_threshold_utils import *
import pickle
import pandas as pd


In [3]:
TEST_DATASET = {
    'trivia_qa': ['trivia_qa','IDK','nq_open', 'pop_qa'],
    'nq_open': ['nq_open','IDK', 'trivia_qa','pop_qa'],
    'pop_qa': ['pop_qa','IDK','nq_open', 'trivia_qa']
}

# , 'ANAH', 'ANAH-v2', 'HaluEval'



In [16]:

dataset = "nq_open"
# label = 'accuracy'
label = 'label'
prompt_type = 'sentence'
filter_refusal = True


for test_dataset in TEST_DATASET[dataset]:
    for fetaure in [f'{prompt_type}_semantic_entropy', f'{prompt_type}_eigen']:
        Feature, Label = load_data(dataset, "val", fetaure, label, prompt_type, filter_refusal=filter_refusal)
        fpr, tpr, thresholds = roc_curve(Label, Feature)
        AUROC = auc(fpr, tpr)
        thresh = get_threshold(thresholds, tpr, fpr)
        # print("AUROC:", AUROC, "thresh", thresh)
        # acc, f1, pre, recall = getAccuracyF1(Label, Feature, thresh, average='macro')
        # acc = round(acc*100, 2)
        # f1 = round(f1*100, 2)
        # pre = round(pre*100, 2)
        # recall = round(recall*100, 2)
        # print("Training:", acc, f1, pre, recall )
        
        output = []
        try:
            test_Feature, test_Label = load_data(test_dataset, 'test', fetaure, label, prompt_type)
            fpr, tpr, _ = roc_curve(test_Label, test_Feature)
            test_AUROC = round(auc(fpr, tpr) * 100, 2)
        except:
            print('NA')
            continue
        output.append(str(test_AUROC))
        acc, f1, pre, recall = getAccuracyF1(test_Label, test_Feature, thresh, average='macro')
        acc = round(acc*100, 2)
        output.append(str(acc))
        f1 = round(f1*100, 2)
        output.append(str(f1))
        pre = round(pre*100, 2)
        output.append(str(pre))
        recall = round(recall*100, 2)
        output.append(str(recall))

        print("\t".join(output))

65.3	60.1	57.33	58.34	60.09
60.95	55.8	53.59	55.55	56.76
66.27	56.8	56.72	58.95	58.65
66.17	64.2	58.38	65.51	60.06
79.21	78.1	70.19	68.63	73.87
69.35	75.1	58.63	59.27	58.21
71.17	64.1	58.44	60.59	66.96
63.65	64.5	55.44	56.26	59.35


In [13]:
import json
for dataset in ['trivia_qa', 'nq_open', 'pop_qa']:
    refusal_rates = []
    for type in ['sentence', 'word']:
        with open(f"/home/ziweiji/Hallu_Det/sem_uncertainty/outputs/{dataset}/{type}/train_refusal_rate.json") as f:
            data = json.load(f)
            refusal_rates.append(round(data['refusal_rate']*100, 2))
    print(dataset+"\t"+"\t".join([str(x) for x in refusal_rates]))

trivia_qa	12.66	0.25
nq_open	15.6	2.42
pop_qa	55.53	2.12


# binary with two features

In [78]:
dataset = "trivia_qa"
label = 'label'
prompt_type = 'sentence'
fetaure = [f'{prompt_type}_semantic_entropy', f'ling_uncertainty']

In [79]:

Feature, Label = load_data(dataset, "val", fetaure, label, prompt_type)
semantic_uncertainty = Feature[:,0]
linguistic_uncertainty = Feature[:,1]
# Sample data: Replace with your actual data
# Assume df is a DataFrame with columns: 'linguistic_uncertainty', 'semantic_uncertainty', 'is_hallucinated'
train_df = pd.DataFrame({
    'semantic_uncertainty': semantic_uncertainty,
    'linguistic_uncertainty': linguistic_uncertainty,
    'is_hallucinated': Label
})


test_Feature, test_Label = load_data(dataset, "test", fetaure, label, prompt_type)
test_df = pd.DataFrame({
    'semantic_uncertainty': test_Feature[:,0],
    'linguistic_uncertainty': test_Feature[:,1],
    'is_hallucinated': test_Label
})



# Define a function to evaluate thresholds
def evaluate_thresholds(linguistic_threshold, semantic_threshold, data):
    predictions = (
        (data['linguistic_uncertainty'] < linguistic_threshold) &
        (data['semantic_uncertainty'] > semantic_threshold)
    ).astype(int)
    
    precision = precision_score(data['is_hallucinated'], predictions, average='macro')
    precision  = round(precision*100, 2)
    recall = recall_score(data['is_hallucinated'], predictions, average='macro')
    recall = round(recall*100, 2)
    f1 = f1_score(data['is_hallucinated'], predictions, average='macro')
    f1 = round(f1*100, 2)
    acc = accuracy_score(data['is_hallucinated'], predictions)
    acc = round(acc*100, 2)
    return precision, recall, f1, acc


In [80]:


# Grid search over possible threshold values
linguistic_thresholds = np.linspace(min(linguistic_uncertainty), max(linguistic_uncertainty), 10)
semantic_thresholds = np.linspace(min(semantic_uncertainty), max(semantic_uncertainty), 10)

best_acc = 0
best_thresholds = (0, 0)
for l_thresh in linguistic_thresholds:
    for s_thresh in semantic_thresholds:
        precision, recall, f1, acc = evaluate_thresholds(l_thresh, s_thresh, train_df)
        
        if acc > best_acc:
            best_acc = acc
            best_thresholds = (l_thresh, s_thresh)

print(f"Best Linguistic Threshold: {best_thresholds[0]}")
print(f"Best Semantic Threshold: {best_thresholds[1]}")
print(f"Best ACC Score: {best_acc}")

# Evaluate on the test set
precision, recall, f1, acc = evaluate_thresholds(best_thresholds[0], best_thresholds[1], test_df)
print("\nTest Set Evaluation:")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"ACC Score: {acc}")

/home/ziweiji/miniconda3/envs/detect/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ziweiji/miniconda3/envs/detect/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ziweiji/miniconda3/envs/detect/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ziweiji/miniconda3/env

Best Linguistic Threshold: 0.5555555555555556
Best Semantic Threshold: 1.5350567286626968
Best ACC Score: 79.8

Test Set Evaluation:
Precision: 68.93
Recall: 57.42
ACC Score: 80.8


/home/ziweiji/miniconda3/envs/detect/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
